In [3]:
import requests
from kafka import KafkaProducer
import json

def send_plane_to_kafka(topic, url, fields={}):

    # Kafka configuration
    kafka_config = {
        'bootstrap_servers': 'kafka1:9092',  # Update with your Kafka broker
    }

    # Initialize Kafka Producer
    producer = KafkaProducer(
        bootstrap_servers=kafka_config['bootstrap_servers'],
        value_serializer=lambda v: json.dumps(v).encode('utf-8')
    )

    # Fetch data from the api    
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        states = data.get("states", [])
        
        # Publish each entry to Kafka
       
        for state in states:
            output = {}
            for key in fields.keys():
                output[key] = state[fields[key]]
            # output["icao24"] = state[0]  # Id of the plane  (ICAO24)
            # output["origin_country"] = state[2]
            # output["velocity"] = state[9]
            # output["timestamp"] = state[3]  # Timestamp de la dernière position
            # output["latitude"] = state[6]  # Latitude de l'avion
            # output["longitude"] = state[5]  # Longitude de l'avion
            producer.send(topic, value=output)
        # Ensure all messages are sent
        producer.flush()
        print(f"Sent {len(states)} records.")
        print(f"Exemple : {states[0]}")
    else:
        print(f"Failed to fetch data: {response.status_code}")

In [4]:
from datetime import datetime, timedelta
fields = {
    "icao24":0,
    "callsign":1,
    "origin_country":2,
    "velocity":9,
    "timestamp":3,
    "latitude":6,
    "longitude":5
}

api_url = "https://opensky-network.org/api/states/all?lamin=30.0&lamax=60.0&lomin=-10&lomax=10.0"

# date initiale
initial_date_str = "2025-01-25 16:00:00"
# durée de récupération (minutes)
step = 60

date_format = "%Y-%m-%d %H:%M:%S"
initial_date = datetime.strptime(initial_date_str, date_format)
end_date = initial_date + timedelta(minutes=step)

start = initial_date.strftime(date_format)
end = end_date.strftime(date_format)

In [ ]:
import time 
for i in range(10):
    print(f"{start}-{end}")        
    send_plane_to_kafka("planes", api_url, fields)
    time.sleep(60)
    initial_date = end_date
    end_date = initial_date + timedelta(minutes=step)
    start = initial_date.strftime(date_format)
    end = end_date.strftime(date_format)

2025-01-25 23:00:00-2025-01-26 00:00:00
Sent 1662 records.
Exemple : ['39d228', 'FHURI1  ', 'France', 1742897059, 1742897059, 7.4556, 43.1859, 1554.48, False, 85.5, 299.96, -2.93, None, 1645.92, None, False, 0]
2025-01-26 00:00:00-2025-01-26 01:00:00
Sent 1657 records.
Exemple : ['39d228', 'FHURI1  ', 'France', 1742897116, 1742897117, 7.4031, 43.208, 1386.84, False, 85.5, 299.96, -2.93, None, 1478.28, None, False, 0]
2025-01-26 01:00:00-2025-01-26 02:00:00
Sent 1660 records.
Exemple : ['39d228', 'FHURI1  ', 'France', 1742897181, 1742897181, 7.3444, 43.2326, 1196.34, False, 85.95, 299.79, -2.93, None, 1287.78, None, False, 0]


In [3]:
import requests
response = requests.get("https://opensky-network.org/api/states/all?lamin=40.0&lamax=52.0&lomin=5&lomax=10.0")

# Vérifier si la requête a réussi
if response.status_code == 200:
    data = response.json()  # Convertir la réponse en dictionnaire
    
    # Vérifier si "states" existe dans les données
    if "states" in data and data["states"]:
        for state in data["states"]:
            icao24 = state[0]  # Identifiant unique de l'avion (ICAO24)
            origin_country = state[2]
            velocity = state[9]
            timestamp = state[3]  # Timestamp de la dernière position
            latitude = state[6]  # Latitude de l'avion
            longitude = state[5]  # Longitude de l'avion

            # Afficher uniquement si latitude et longitude sont disponibles
            if latitude is not None and longitude is not None:
                print(f"ID: {icao24}, Timestamp: {timestamp}, Latitude: {latitude}, Longitude: {longitude}, Velocity : {velocity}, Origin : {origin_country}")
    else:
        print("Aucune donnée d'avion disponible.")
else:
    print("Erreur :", response.text)

ID: 39d228, Timestamp: 1742895328, Latitude: 42.59, Longitude: 8.7473, Velocity : 48.02, Origin : France
ID: 4b1800, Timestamp: 1742895330, Latitude: 48.9363, Longitude: 9.8733, Velocity : 248.62, Origin : Switzerland
ID: 4b181a, Timestamp: 1742895330, Latitude: 46.6728, Longitude: 7.0913, Velocity : 184.07, Origin : Switzerland
ID: 4952c1, Timestamp: 1742895330, Latitude: 43.6425, Longitude: 7.4365, Velocity : 232.86, Origin : Portugal
ID: 440db0, Timestamp: 1742895313, Latitude: 47.596, Longitude: 7.5297, Velocity : 0, Origin : Austria
ID: 39de44, Timestamp: 1742895330, Latitude: 42.5615, Longitude: 8.0671, Velocity : 236.55, Origin : France
ID: 4b180b, Timestamp: 1742895329, Latitude: 48.6892, Longitude: 9.1951, Velocity : 0, Origin : Switzerland
ID: 4bc8c7, Timestamp: 1742895315, Latitude: 50.0487, Longitude: 8.589, Velocity : 0, Origin : Turkey
ID: 511172, Timestamp: 1742895331, Latitude: 46.0892, Longitude: 5.3901, Velocity : 249.38, Origin : Estonia
ID: 3c6672, Timestamp: 174289